In [1]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Mahesh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import warnings
warnings.filterwarnings('ignore')

In [3]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

# Preprocessing

In [4]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read() 
intents = json.loads(data_file)

In [5]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [6]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Mahesh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))
print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique lemmatized words", words)
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

360 documents
49 classes ['SWO', 'Sports Secretary', 'admission', 'canteen', 'challan', 'civil_engineering_info', 'college intake', 'committee', 'computer_science_info', 'computerhod', 'course', 'creator', 'cs_staffs', 'document', 'electrical_staffs', 'electronics_and_communication_info', 'email', 'event', 'facilities', 'fees', 'floors', 'goodbye', 'greeting', 'hod_ce', 'hod_contact', 'hod_cse', 'hod_ec', 'hod_me', 'hostel', 'hours', 'infrastructure', 'library', 'location', 'mechanical_engineering_info', 'mechanical_staffs', 'menu', 'name', 'number', 'placement', 'principal', 'ragging', 'random', 'salutaion', 'scholarship', 'staff', 'swear', 'syllabus', 'task', 'uniform']
254 unique lemmatized words ["'s", 'a', 'about', 'active', 'address', 'administrative', 'admision', 'admission', 'album', 'allotment', 'am', 'an', 'and', 'antiragging', 'any', 'anyone', 'are', 'as', 'assistant', 'at', 'attend', 'available', 'average', 'between', 'big', 'bitch', 'book', 'branch', 'bring', 'building', '

In [8]:
import random
import numpy as np
training = []
output_empty = [0] * len(classes)


for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    output_row = list(output_empty)
    
    output_row[classes.index(doc[1])] = 1
    training.append((bag, output_row))

random.shuffle(training)


train_x = np.array([t[0] for t in training])
train_y = np.array([t[1] for t in training])

print("Training data created")

Training data created


In [9]:
from tensorflow.python.framework import ops
ops.reset_default_graph()

In [10]:
from keras.models import Sequential
from keras.layers import Dense,Dropout
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [11]:
from tensorflow.keras.optimizers import SGD
sgd = SGD(decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [12]:
hist = model.fit(np.array(train_x), np.array(train_y), epochs=220, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")


Epoch 1/220
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0361 - loss: 3.8639
Epoch 2/220
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0839 - loss: 3.6544
Epoch 3/220
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1103 - loss: 3.5271
Epoch 4/220
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1876 - loss: 3.3062
Epoch 5/220
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2187 - loss: 3.1914
Epoch 6/220
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.2968 - loss: 2.7576   
Epoch 7/220
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2984 - loss: 2.6860
Epoch 8/220
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3777 - loss: 2.4055
Epoch 9/220
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4178 - loss: 2.2030
Epoch 10/220
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4551 - loss: 2.0992
Epoch 11/220
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4818 - loss: 1.8433
Epoch 12/220
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accura

model created


In [13]:
from keras.models import load_model
model = load_model('chatbot_model.h5')

In [15]:
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [16]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):    
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [17]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res

In [18]:
import numpy as np
import random
a="hi"
print(type(a))
chatbot_response(a)


<class 'str'>
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step


'Hi there, how can I help?'

In [19]:
res=chatbot_response('what is your name')
print(res)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
You can call me GPTCBOT.


In [20]:
a=str(input("ask me?"))
res=chatbot_response(a)
print(a)
print(res)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step


KeyError: 'responses'